In [ ]:
# # Question Answering

# Idea-1 SQL-based model
# SQL-based: Develop Text-to-Sql generation system with Google Gemini Pro LLM (Colab-based code) (UI-based code will be done if goes with this)

# Idea-2 RAG based model with CSV data
# RAG: Build Chatbot with CSV data (RAG) with Open source Mistral LLM and LangChain framework (Colab-based code) (UI-based code will be done if goes with this)

# Idea-3 pre-train model on your own data TAPAS model (UI-based code will be done if goes with this)

In [ ]:
# Dataset: https://www.kaggle.com/datasets/prathammalvia/imdb-sqlite-dataset

In [ ]:
# Idea-1 AgenticAI and SQL-based model
# SQL-based: Develop Text-to-Sql generation system with Google Gemini Pro LLM (Colab-based code)

In [ ]:
!pip install -q -U langchain==0.1.2
!pip install -q -U google-generativeai==0.5.2
!pip install -q -U langchain-google-genai==1.0.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 55.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GEMINI_KEY')

In [ ]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:////content/movies.sqlite")
print(db.dialect)
print(db.get_usable_table_names())

sqlite
['directors', 'movies']


In [ ]:
print("Q1: ")
db.run("SELECT * FROM directors LIMIT 1;")
# columns=['name', 'id', 'gender', 'uid', 'department']

Q1: 


"[('James Cameron', 4762, 2, 2710, 'Directing')]"

In [ ]:
print("Q2: ")
db.run("SELECT * FROM movies LIMIT 1;")
# columns=['id', 'original_title', 'budget', 'popularity', 'release_date',
# 'revenue', 'title', 'vote_average', 'vote_count', 'overview', 'tagline',
# 'uid', 'director_id' ]

Q2: 


"[(43597, 'Avatar', 237000000, 150, '2009-12-10', 2787965087, 'Avatar', 7.2, 11800, 'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.', 'Enter the World of Pandora.', 19995, 4762)]"

In [ ]:
import re
from langchain.chains import create_sql_query_chain
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=GOOGLE_API_KEY, convert_system_message_to_human=True, temperature=0.0)
chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many directors are men"})
response

/usr/local/lib/python3.10/dist-packages/langchain_google_genai/chat_models.py:326: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


'```sql\nSELECT COUNT(*) AS "Number of Male Directors"\nFROM directors\nWHERE gender = 2;\n```'

In [ ]:

# Generate regex pattern to remove prefix and suffix
regex_pattern = r'^```sql\n|\n```$'

# Remove prefix and suffix using regex to get proper SQL query
modified_response = re.sub(regex_pattern, '', response)
print(modified_response)

SELECT COUNT(*) AS "Number of Male Directors"
FROM directors
WHERE gender = 2;


In [ ]:
db.run(modified_response)

'[(1574,)]'

In [ ]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

In [ ]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
"""Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
{query}
SQL Result: {result}
Answer: """
)

answer = answer_prompt | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

chain.invoke({"question": "How many directors are men"})

/usr/local/lib/python3.10/dist-packages/langchain_google_genai/chat_models.py:326: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
/usr/local/lib/python3.10/dist-packages/langchain_google_genai/chat_models.py:326: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


'There are 1574 male directors.'

In [ ]:
chain.invoke({'question': "How many movie relaesed in year 2010? Which movie has the most rating and who directed in 2010?"})

/usr/local/lib/python3.10/dist-packages/langchain_google_genai/chat_models.py:326: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
/usr/local/lib/python3.10/dist-packages/langchain_google_genai/chat_models.py:326: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


'In 2010, there was only 1 movie released. The movie with the highest rating is 8.1, which was directed by Christopher Nolan.'

In [ ]:
chain.invoke({'question': "List all the total movie of year 2010."})

/usr/local/lib/python3.10/dist-packages/langchain_google_genai/chat_models.py:326: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
/usr/local/lib/python3.10/dist-packages/langchain_google_genai/chat_models.py:326: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


'In the year 2010, there were 223 movies released.'

In [ ]:
chain.invoke({'question': "List the top 5 movie of 2010 with highest rating?"})

/usr/local/lib/python3.10/dist-packages/langchain_google_genai/chat_models.py:326: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
/usr/local/lib/python3.10/dist-packages/langchain_google_genai/chat_models.py:326: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


'The top 5 movies of 2010 with the highest ratings are:\n1. Inception (8.1)\n2. Incendies (7.9)\n3. Shutter Island (7.8)\n4. My Name Is Khan (7.7)\n5. Inside Job (7.7)'

In [ ]:
chain.invoke({'question': "Which directors directed 5 highest rated movies of year 2010 (also add name and rating of the movie)?"})

/usr/local/lib/python3.10/dist-packages/langchain_google_genai/chat_models.py:326: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
/usr/local/lib/python3.10/dist-packages/langchain_google_genai/chat_models.py:326: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


'The five highest rated movies of 2010 are:\n- Inception, directed by Christopher Nolan, with a rating of 8.1\n- Incendies, directed by Denis Villeneuve, with a rating of 7.9\n- Shutter Island, directed by Martin Scorsese, with a rating of 7.8\n- My Name Is Khan, directed by Karan Johar, with a rating of 7.7\n- Inside Job, directed by Charles Ferguson, with a rating of 7.7'

In [ ]:
# Idea-2 RAG based model with CSV data
# RAG: Build Chatbot (RAG) on your local Laptop using Open Source Mistral LLM and LangChain framework (Colab-based code)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

In [ ]:
db = '/content/movies.sqlite'
sqliteConnection = sqlite3.connect(db)
cur = sqliteConnection.cursor()

In [ ]:
# Get all the data about movies
query='SELECT * FROM movies'
cur.execute(query)
movies = cur.fetchall()

# Displaying the movies data
movies[:3]

[(43597,
  'Avatar',
  237000000,
  150,
  '2009-12-10',
  2787965087,
  'Avatar',
  7.2,
  11800,
  'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.',
  'Enter the World of Pandora.',
  19995,
  4762),
 (43598,
  "Pirates of the Caribbean: At World's End",
  300000000,
  139,
  '2007-05-19',
  961000000,
  "Pirates of the Caribbean: At World's End",
  6.9,
  4500,
  'Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of the Earth with Will Turner and Elizabeth Swann. But nothing is quite as it seems.',
  'At the end of the world, the adventure begins.',
  285,
  4763),
 (43599,
  'Spectre',
  245000000,
  107,
  '2015-10-26',
  880674609,
  'Spectre',
  6.3,
  4466,
  'A cryptic message from Bond’s past sends him on a trail to uncover a sinister organization. While M battles political forces to keep the secret service 

In [ ]:
query='SELECT * FROM directors'
cur.execute(query)
directors=cur.fetchall()

# Displaying the directors data
directors[:3]

[('James Cameron', 4762, 2, 2710, 'Directing'),
 ('Gore Verbinski', 4763, 2, 1704, 'Directing'),
 ('Sam Mendes', 4764, 2, 39, 'Directing')]

In [ ]:
# Creating a movies DataFrame
movies=pd.DataFrame(movies, columns=['id', 'original_title', 'budget', 'popularity', 'release_date',
'revenue', 'title', 'vote_average', 'vote_count', 'overview', 'tagline',
'uid', 'director_id' ])

# Displaying the movies DataFrame
movies.head()

,id,original_title,budget,popularity,release_date,revenue,title,vote_average,vote_count,overview,tagline,uid,director_id
0,43597,Avatar,237000000,150,2009-12-10,2787965087,Avatar,7.2,11800,"In the 22nd century, a paraplegic Marine is di...",Enter the World of Pandora.,19995,4762
1,43598,Pirates of the Caribbean: At World's End,300000000,139,2007-05-19,961000000,Pirates of the Caribbean: At World's End,6.9,4500,"Captain Barbossa, long believed to be dead, ha...","At the end of the world, the adventure begins.",285,4763
2,43599,Spectre,245000000,107,2015-10-26,880674609,Spectre,6.3,4466,A cryptic message from Bond’s past sends him o...,A Plan No One Escapes,206647,4764
3,43600,The Dark Knight Rises,250000000,112,2012-07-16,1084939099,The Dark Knight Rises,7.6,9106,Following the death of District Attorney Harve...,The Legend Ends,49026,4765
4,43601,John Carter,260000000,43,2012-03-07,284139100,John Carter,6.1,2124,"John Carter is a war-weary, former military ca...","Lost in our world, found in another.",49529,4766


In [ ]:
movies.columns

Index(['id', 'original_title', 'budget', 'popularity', 'release_date',
       'revenue', 'title', 'vote_average', 'vote_count', 'overview', 'tagline',
       'uid', 'director_id'],
      dtype='object')

In [ ]:
import os
os.mkdir('data')

In [ ]:
movies.to_csv("data/movies.csv", index=False)

In [ ]:
# Creating a directors DataFrame
directors=pd.DataFrame(directors, columns=['name', 'id', 'gender', 'uid', 'department'])

# Displaying the directors DataFrame
directors.head()

,name,id,gender,uid,department
0,James Cameron,4762,2,2710,Directing
1,Gore Verbinski,4763,2,1704,Directing
2,Sam Mendes,4764,2,39,Directing
3,Christopher Nolan,4765,2,525,Directing
4,Andrew Stanton,4766,2,7,Directing


In [ ]:
directors.columns

Index(['name', 'id', 'gender', 'uid', 'department'], dtype='object')

In [ ]:
directors.to_csv("data/directors.csv", index=False)

In [ ]:
# Huggingface libraries to run LLM.
!pip install -q -U transformers==4.40.2
!pip install -q -U accelerate==0.30.1
!pip install -q -U bitsandbytes==0.43.1
!pip install -q -U huggingface_hub==0.23.0

#LangChain related libraries
!pip install -q -U langchain==0.1.2

#Open-source pure-python PDF library capable of splitting, merging, cropping,
#and transforming the pages of PDF files
!pip install -q -U pypdf==4.2.0

#Python framework for state-of-the-art sentence, text and image embeddings.
!pip install -q -U sentence-transformers==2.7.0

# FAISS Vector Databses specific Libraries
!pip install -q -U faiss-gpu==1.7.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from huggingface_hub import login
from google.colab import userdata

login(token=userdata.get('HF_KEY'))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
#from typing import List
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig, BitsAndBytesConfig
import torch
from langchain.llms import HuggingFacePipeline

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFaceHub
from langchain.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain

Device: cuda
Tesla T4


In [ ]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print("Device:", device)
if device == 'cuda':
    print(torch.cuda.get_device_name(0))

Device: cuda
Tesla T4


In [ ]:
origin_model_path = "mistralai/Mistral-7B-Instruct-v0.1"
model_path = "filipealmeida/Mistral-7B-Instruct-v0.1-sharded"
bnb_config = BitsAndBytesConfig \
              (
                load_in_4bit=True,
                bnb_4bit_use_double_quant=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_compute_dtype=torch.bfloat16,
              )
model = AutoModelForCausalLM.from_pretrained (model_path, trust_remote_code=True,
                                              quantization_config=bnb_config,
                                              device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(origin_model_path)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=300,
    temperature = 0.3,
    do_sample=True,
)
mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.document_loaders import DirectoryLoader
# loader = DirectoryLoader('./data', glob='**/*.csv', loader_cls=CSVLoader)
loader = CSVLoader('./data/movies.csv')
data = loader.load()

# Split the documents into smaller chunks
#separator=""
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
chunked_docs  = text_splitter.split_documents(data)

# Using HuggingFace embeddings
embeddings = HuggingFaceEmbeddings()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from langchain.vectorstores import FAISS
db = FAISS.from_documents(chunked_docs,
                          HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))


# Connect query to FAISS index using a retriever
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 4}
)

In [ ]:
# Create the Conversational Retrieval Chain
qa_chain = ConversationalRetrievalChain.from_llm(mistral_llm, retriever,return_source_documents=True)

In [ ]:
import sys
chat_history = []

In [ ]:
while True:
  query = input('Prompt: ')
  # Ask question related CSV file like Which country has highest green gas?
  if query.lower().strip()=="exit":
    break
  result = qa_chain.invoke({'question': query, 'chat_history': chat_history})

  print('Answer: ' + result['answer'] + '\n')
  chat_history.append((query, result['answer']))

Prompt: When was Inception movie released and give me all details related to this.
Answer:  The Inception movie was released on July 14th, 2010. It had a budget of $160 million and made $825.5 million in revenue. It received an average rating of 8.1 out of 10 based on 13,752 votes.

Prompt: Which movie has low rating than Inception released around same time? 
Answer:   Based on the information provided, there are two movies that were released around the same time as Inception and have a lower rating than it. These movies are Final Destination and Identity.

Prompt: What are the ratings of these movies?
Answer:   The rating of Final Destination is 6.4 and the rating of Identity is 7.1.

Prompt: When was these all movies were released and  what was tagline?
Answer:   The release date for Final Destination was 2000-03-16 and its tagline was "No accidents. No coincidences. No escapes. You can't cheat death." The release date for Identity was 2003-04-25 and its tagline was "The secret lies 

In [ ]:
chat_history

[('When was Inception movie released and give me all details related to this.',
  ' The Inception movie was released on July 14th, 2010. It had a budget of $160 million and made $825.5 million in revenue. It received an average rating of 8.1 out of 10 based on 13,752 votes.'),
 ('Which movie has low rating than Inception released around same time? ',
  '  Based on the information provided, there are two movies that were released around the same time as Inception and have a lower rating than it. These movies are Final Destination and Identity.'),
 ('What are the ratings of these movies?',
  '  The rating of Final Destination is 6.4 and the rating of Identity is 7.1.'),
 ('When was these all movies were released and  what was tagline?',
  '  The release date for Final Destination was 2000-03-16 and its tagline was "No accidents. No coincidences. No escapes. You can\'t cheat death." The release date for Identity was 2003-04-25 and its tagline was "The secret lies within."'),
 ('What was t

In [ ]:
while True:
  query = input('Prompt: ')
  # Ask question related CSV file like Which country has highest green gas?
  if query.lower().strip()=="exit":
    break
  result = qa_chain.invoke({'question': query, 'chat_history': chat_history})

  print('Answer: ' + result['answer'] + '\n')
  chat_history.append((query, result['answer']))

Prompt: Give me all details about "Inception" movie.
Answer:  

The movie "Inception" was directed by Christopher Nolan and released in 2010. It starred Leonardo DiCaprio, Joseph Gordon-Levitt, Ellen Page, Tom Hardy, Ken Watanabe, Cillian Murphy, Tom Berenger, and Marion Cotillard. The movie had a budget of $160 million and made $825.5 million at the box office. It received an average rating of 8.1 out of 10 from critics and a 74% approval rating based on reviews from Rotten Tomatoes. The movie is about a skilled thief named Cobb who is offered a chance to regain his old life as payment for a task considered to be impossible: "inception", the implantation of another person's idea into a target's subconscious.

Prompt: exit


In [ ]:
chat_history

[('When was Inception movie released and give me all details related to this.',
  ' The Inception movie was released on July 14th, 2010. It had a budget of $160 million and made $825.5 million in revenue. It received an average rating of 8.1 out of 10 based on 13,752 votes.'),
 ('Which movie has low rating than Inception released around same time? ',
  '  Based on the information provided, there are two movies that were released around the same time as Inception and have a lower rating than it. These movies are Final Destination and Identity.'),
 ('What are the ratings of these movies?',
  '  The rating of Final Destination is 6.4 and the rating of Identity is 7.1.'),
 ('When was these all movies were released and  what was tagline?',
  '  The release date for Final Destination was 2000-03-16 and its tagline was "No accidents. No coincidences. No escapes. You can\'t cheat death." The release date for Identity was 2003-04-25 and its tagline was "The secret lies within."'),
 ('What was t

In [ ]:
# Idea-3 pre-train model on your own data TAPAS model

In [ ]:
# paper: https://aclanthology.org/2020.acl-main.398.pdf

In [ ]:
# Refer to learn more: https://colab.research.google.com/github/NielsRogge/Transformers-Tutorials/blob/master/TAPAS/Fine_tuning_TapasForQuestionAnswering_on_SQA.ipynb

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

In [ ]:
db = '/content/movies.sqlite'
sqliteConnection = sqlite3.connect(db)
cur = sqliteConnection.cursor()

In [ ]:
# Get all the data about movies
query='SELECT * FROM movies'
cur.execute(query)
movies = cur.fetchall()

# Displaying the movies data
movies[:3]

[(43597,
  'Avatar',
  237000000,
  150,
  '2009-12-10',
  2787965087,
  'Avatar',
  7.2,
  11800,
  'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.',
  'Enter the World of Pandora.',
  19995,
  4762),
 (43598,
  "Pirates of the Caribbean: At World's End",
  300000000,
  139,
  '2007-05-19',
  961000000,
  "Pirates of the Caribbean: At World's End",
  6.9,
  4500,
  'Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of the Earth with Will Turner and Elizabeth Swann. But nothing is quite as it seems.',
  'At the end of the world, the adventure begins.',
  285,
  4763),
 (43599,
  'Spectre',
  245000000,
  107,
  '2015-10-26',
  880674609,
  'Spectre',
  6.3,
  4466,
  'A cryptic message from Bond’s past sends him on a trail to uncover a sinister organization. While M battles political forces to keep the secret service 

In [ ]:
query='SELECT * FROM directors'
cur.execute(query)
directors=cur.fetchall()

# Displaying the directors data
directors[:3]

[('James Cameron', 4762, 2, 2710, 'Directing'),
 ('Gore Verbinski', 4763, 2, 1704, 'Directing'),
 ('Sam Mendes', 4764, 2, 39, 'Directing')]

In [ ]:
# Creating a movies DataFrame
movies=pd.DataFrame(movies, columns=['id', 'original_title', 'budget', 'popularity', 'release_date',
'revenue', 'title', 'vote_average', 'vote_count', 'overview', 'tagline',
'uid', 'director_id' ])

# Displaying the movies DataFrame
movies.head()

,id,original_title,budget,popularity,release_date,revenue,title,vote_average,vote_count,overview,tagline,uid,director_id
0,43597,Avatar,237000000,150,2009-12-10,2787965087,Avatar,7.2,11800,"In the 22nd century, a paraplegic Marine is di...",Enter the World of Pandora.,19995,4762
1,43598,Pirates of the Caribbean: At World's End,300000000,139,2007-05-19,961000000,Pirates of the Caribbean: At World's End,6.9,4500,"Captain Barbossa, long believed to be dead, ha...","At the end of the world, the adventure begins.",285,4763
2,43599,Spectre,245000000,107,2015-10-26,880674609,Spectre,6.3,4466,A cryptic message from Bond’s past sends him o...,A Plan No One Escapes,206647,4764
3,43600,The Dark Knight Rises,250000000,112,2012-07-16,1084939099,The Dark Knight Rises,7.6,9106,Following the death of District Attorney Harve...,The Legend Ends,49026,4765
4,43601,John Carter,260000000,43,2012-03-07,284139100,John Carter,6.1,2124,"John Carter is a war-weary, former military ca...","Lost in our world, found in another.",49529,4766


In [ ]:
movies.columns

Index(['id', 'original_title', 'budget', 'popularity', 'release_date',
       'revenue', 'title', 'vote_average', 'vote_count', 'overview', 'tagline',
       'uid', 'director_id'],
      dtype='object')

In [ ]:
import os
os.mkdir('data')

In [ ]:
movies.to_csv("data/movies.csv", index=False)

In [ ]:
# Creating a directors DataFrame
directors=pd.DataFrame(directors, columns=['name', 'id', 'gender', 'uid', 'department'])

# Displaying the directors DataFrame
directors.head()

,name,id,gender,uid,department
0,James Cameron,4762,2,2710,Directing
1,Gore Verbinski,4763,2,1704,Directing
2,Sam Mendes,4764,2,39,Directing
3,Christopher Nolan,4765,2,525,Directing
4,Andrew Stanton,4766,2,7,Directing


In [ ]:
directors.columns

Index(['name', 'id', 'gender', 'uid', 'department'], dtype='object')

In [ ]:
directors.to_csv("data/directors.csv", index=False)

In [ ]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print("Device:", device)
if device == 'cuda':
    print(torch.cuda.get_device_name(0))
CUDA = torch.version.cuda
print(torch.version.cuda)

Device: cpu
12.1


In [ ]:
!pip install transformers

In [ ]:
# !pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.6.0+${CUDA}.html

In [ ]:
from transformers import TapasTokenizer, TapasForQuestionAnswering
import pandas as pd

In [ ]:
# Define the table
data = pd.read_csv('./data/movies.csv')
data.head()

,id,original_title,budget,popularity,release_date,revenue,title,vote_average,vote_count,overview,tagline,uid,director_id
0,43597,Avatar,237000000,150,2009-12-10,2787965087,Avatar,7.2,11800,"In the 22nd century, a paraplegic Marine is di...",Enter the World of Pandora.,19995,4762
1,43598,Pirates of the Caribbean: At World's End,300000000,139,2007-05-19,961000000,Pirates of the Caribbean: At World's End,6.9,4500,"Captain Barbossa, long believed to be dead, ha...","At the end of the world, the adventure begins.",285,4763
2,43599,Spectre,245000000,107,2015-10-26,880674609,Spectre,6.3,4466,A cryptic message from Bond’s past sends him o...,A Plan No One Escapes,206647,4764
3,43600,The Dark Knight Rises,250000000,112,2012-07-16,1084939099,The Dark Knight Rises,7.6,9106,Following the death of District Attorney Harve...,The Legend Ends,49026,4765
4,43601,John Carter,260000000,43,2012-03-07,284139100,John Carter,6.1,2124,"John Carter is a war-weary, former military ca...","Lost in our world, found in another.",49529,4766


In [ ]:
# new_data = data.to_dict('list')
# pprint(new_data)
for i in data.columns:
  data[i] = data[i].astype('str')

In [ ]:
queries = ["When was Avatar movie released?",
           "What is the budget of Spectre?",
           "What is the vote average of The Dark Knight Rises?",
           "What is the tagline of Pirates of the Caribbean: At World's End?"]

In [ ]:
# # Define the questions
# queries = ["When was Inception movie released?",
#            "Which movie has low rating than Inception released around same time?",
#            "What is the ratings of Final Destination?",
#            "What is the ratings of Identity?",
#            "What is the tagline of Inception?"]
# queries

['When was Inception movie released?',
 'Which movie has low rating than Inception released around same time?',
 'What is the ratings of Final Destination?',
 'What is the ratings of Identity?',
 'What is the tagline of Inception?']

In [ ]:
# # Define the table
# data = {'Cities': ["Paris, France", "London, England", "Lyon, France"], 'Inhabitants': ["2.161", "8.982", "0.513"]}

# # Define the questions
# queries = ["Which city has most inhabitants?", "What is the average number of inhabitants?", "How many French cities are in the list?", "How many inhabitants live in French cities?"]


In [ ]:
def load_model_and_tokenizer():
  """
    Load
  """
  # Load pretrained tokenizer: TAPAS finetuned on WikiTable Questions
  tokenizer = TapasTokenizer.from_pretrained("google/tapas-base-finetuned-wtq")

  # Load pretrained model: TAPAS finetuned on WikiTable Questions
  model = TapasForQuestionAnswering.from_pretrained("google/tapas-base-finetuned-wtq")

  # Return tokenizer and model
  return tokenizer, model

In [ ]:
def prepare_inputs(data, queries, tokenizer):
  """
    Convert dictionary into data frame and tokenize inputs given queries.
  """
  # Prepare inputs
  table = pd.DataFrame.from_dict(data)
  # table = data
  inputs = tokenizer(table=table, queries=queries, padding='max_length', truncation=True,  return_tensors="pt")

  # Return things
  return table, inputs

In [ ]:
def generate_predictions(inputs, model, tokenizer):
  """
    Generate predictions for some tokenized input.
  """
  # Generate model results
  outputs = model(**inputs)

  # Convert logit outputs into predictions for table cells and aggregation operators
  predicted_table_cell_coords, predicted_aggregation_operators = tokenizer.convert_logits_to_predictions(
          inputs,
          outputs.logits.detach(),
          outputs.logits_aggregation.detach()
  )

  # Return values
  return predicted_table_cell_coords, predicted_aggregation_operators

In [ ]:
def postprocess_predictions(predicted_aggregation_operators, predicted_table_cell_coords, table):
  """
    Compute the predicted operation and nicely structure the answers.
  """
  # Process predicted aggregation operators
  aggregation_operators = {0: "NONE", 1: "SUM", 2: "AVERAGE", 3:"COUNT"}
  aggregation_predictions_string = [aggregation_operators[x] for x in predicted_aggregation_operators]

  # Process predicted table cell coordinates
  answers = []
  for coordinates in predicted_table_cell_coords:
    if len(coordinates) == 1:
      # 1 cell
      answers.append(table.iat[coordinates[0]])
    else:
      # > 1 cell
      cell_values = []
      for coordinate in coordinates:
        cell_values.append(table.iat[coordinate])
      answers.append(", ".join(cell_values))

  # Return values
  return aggregation_predictions_string, answers

In [ ]:
def show_answers(queries, answers, aggregation_predictions_string):
  """
    Visualize the postprocessed answers.
  """
  for query, answer, predicted_agg in zip(queries, answers, aggregation_predictions_string):
    print(query)
    if predicted_agg == "NONE":
      print("Predicted answer: " + answer)
    else:
      print("Predicted answer: " + predicted_agg + " > " + answer)

In [ ]:
def run_tapas(data):
  """
    Invoke the TAPAS model.
  """
  tokenizer, model = load_model_and_tokenizer()
  table, inputs = prepare_inputs(data, queries, tokenizer)
  predicted_table_cell_coords, predicted_aggregation_operators = generate_predictions(inputs, model, tokenizer)
  aggregation_predictions_string, answers = postprocess_predictions(predicted_aggregation_operators, predicted_table_cell_coords, table)
  show_answers(queries, answers, aggregation_predictions_string)

In [ ]:
run_tapas(data[:4].to_dict())

When was Avatar movie released?
Predicted answer: 2009-12-10
What is the budget of Spectre?
Predicted answer: 245000000
What is the vote average of The Dark Knight Rises?
Predicted answer: AVERAGE > 7.6
What is the tagline of Pirates of the Caribbean: At World's End?
Predicted answer: At the end of the world, the adventure begins.
